# 📚 日本語小説 → MP3 変換ツール
## OpenAI TTS API を使用して日本語小説を音声化

このノートブックでは、5000文字程度の日本語小説をOpenAI TTS APIで音声化します。

### 📋 機能
- 🔑 Colabシークレットから安全にAPIキー取得
- 📁 テキストファイルのアップロード・自動エンコーディング検出
- 📦 段落単位でのスマート分割（4000文字以内）
- 🎙️ 6種類の音声から選択可能
- 📊 リアルタイム進捗表示
- 🔗 全チャンクを1つのMP3に結合

---
## セル1: 📦 必要なライブラリのインストール

In [ ]:
!pip install openai pydub tqdm -q
!apt-get install -y ffmpeg -qq

print("✅ ライブラリのインストール完了！")

---
## セル2: 🔧 ライブラリのインポートとAPIキー取得

⚠️ **事前準備**: 左サイドバーの 🔑（シークレット）から `OPENAI_API_KEY` を追加してください

In [ ]:
import os
import time
import re
from pathlib import Path
from google.colab import userdata, files
from openai import OpenAI
from pydub import AudioSegment
import tempfile
from tqdm.notebook import tqdm
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Colabシークレットから API キーを取得
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    if not OPENAI_API_KEY:
        raise ValueError("API キーが空です")
    print("✅ OpenAI API キーを取得しました")
except Exception as e:
    print(f"❌ エラー: Colabシークレットから 'OPENAI_API_KEY' を取得できませんでした")
    print("📝 設定方法: 左サイドバー 🔑 → 'OPENAI_API_KEY' を追加")
    raise SystemExit("API キーを設定してから再実行してください")

# OpenAI クライアントの初期化
client = OpenAI(api_key=OPENAI_API_KEY)
print("✅ OpenAI クライアント初期化完了")

---
## セル3: 🎙️ 音声タイプとモデルの選択

お好みの音声を選択してください：

| 音声 | 特徴 |
|------|------|
| 🎭 Alloy | 中性的でバランスの取れた、落ち着いた声 |
| 🎤 Echo | 落ち着いた、やや深みのある男性的な声 |
| 📖 Fable | 表現力豊かで、物語の朗読に向いた声 |
| 💪 Onyx | 力強く、自信に満ちた男性的な声 |
| ✨ Nova | 明るく、親しみやすい女性的な声 |
| 💎 Shimmer | 澄んだ、知的な印象を与える女性的な声 |

In [ ]:
# 利用可能な音声オプション
VOICE_OPTIONS = {
    "alloy": "🎭 Alloy - 中性的でバランスの取れた、落ち着いた声",
    "echo": "🎤 Echo - 落ち着いた、やや深みのある男性的な声",
    "fable": "📖 Fable - 表現力豊かで、物語の朗読に向いた声",
    "onyx": "💪 Onyx - 力強く、自信に満ちた男性的な声",
    "nova": "✨ Nova - 明るく、親しみやすい女性的な声",
    "shimmer": "💎 Shimmer - 澄んだ、知的な印象を与える女性的な声"
}

# ドロップダウンで選択
voice_dropdown = widgets.Dropdown(
    options=[(desc, voice_id) for voice_id, desc in VOICE_OPTIONS.items()],
    value='fable',  # 小説朗読向けのFableをデフォルトに
    description='音声:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# モデル選択
model_dropdown = widgets.Dropdown(
    options=[
        ('tts-1 (標準品質・高速)', 'tts-1'),
        ('tts-1-hd (高品質)', 'tts-1-hd')
    ],
    value='tts-1',
    description='モデル:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# 設定値を保存
config = {'voice': 'fable', 'model': 'tts-1', 'confirmed': False}

# 選択確認ボタン
confirm_button = widgets.Button(
    description='✅ この設定で続行',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

status_label = widgets.HTML(value="")

def on_confirm_click(b):
    config['voice'] = voice_dropdown.value
    config['model'] = model_dropdown.value
    config['confirmed'] = True
    status_label.value = f"""
    <div style='padding: 10px; background-color: #d4edda; border-radius: 5px; margin-top: 10px;'>
        <b>✅ 設定完了!</b><br>
        🎙️ 音声: {VOICE_OPTIONS[config['voice']]}<br>
        📀 モデル: {config['model']}
    </div>
    """

confirm_button.on_click(on_confirm_click)

# ウィジェットを表示
display(widgets.VBox([
    widgets.HTML("<h3>🎛️ 音声設定</h3>"),
    voice_dropdown,
    model_dropdown,
    widgets.HTML("<br>"),
    confirm_button,
    status_label
]))

---
## セル4: 📁 テキストファイルのアップロード

In [ ]:
# 設定値を取得
TTS_VOICE = config['voice']
TTS_MODEL = config['model']
MAX_CHUNK_SIZE = 4000
REQUEST_INTERVAL = 0.5

print(f"📋 使用する設定:")
print(f"   🎙️ 音声: {VOICE_OPTIONS[TTS_VOICE]}")
print(f"   📀 モデル: {TTS_MODEL}")
print(f"   📏 最大チャンクサイズ: {MAX_CHUNK_SIZE}文字")
print("\n" + "="*50)
print("📁 テキストファイルをアップロードしてください（.txt形式）")
print("="*50 + "\n")

uploaded = files.upload()

if not uploaded:
    raise ValueError("❌ ファイルがアップロードされませんでした")

# アップロードされたファイルを読み込み
filename = list(uploaded.keys())[0]
print(f"\n📄 アップロードされたファイル: {filename}")

# エンコーディングを自動検出して読み込み
encodings_to_try = ['utf-8', 'shift_jis', 'cp932', 'euc-jp']
novel_text = None

for encoding in encodings_to_try:
    try:
        novel_text = uploaded[filename].decode(encoding)
        print(f"✅ エンコーディング '{encoding}' で読み込み成功")
        break
    except UnicodeDecodeError:
        continue

if novel_text is None:
    raise ValueError("❌ ファイルのエンコーディングを検出できませんでした")

# テキストの前処理
novel_text = novel_text.strip()
novel_text = re.sub(r'\r\n', '\n', novel_text)
novel_text = re.sub(r'\n{3,}', '\n\n', novel_text)

print(f"\n📊 テキスト情報:")
print(f"   📝 総文字数: {len(novel_text):,} 文字")
print(f"   📑 段落数: {len([p for p in novel_text.split(chr(10)+chr(10)) if p.strip()])} 段落")

---
## セル5: 📦 テキストを段落単位で分割

In [ ]:
def split_text_into_chunks(text: str, max_size: int = 4000) -> list:
    """
    テキストを段落単位で分割し、各チャンクが max_size 以下になるようにする
    """
    paragraphs = text.split('\n\n')
    paragraphs = [p.strip() for p in paragraphs if p.strip()]
    
    chunks = []
    current_chunk = ""
    
    for paragraph in paragraphs:
        if len(paragraph) > max_size:
            if current_chunk:
                chunks.append(current_chunk.strip())
                current_chunk = ""
            
            sentences = re.split(r'([。！？])', paragraph)
            temp_chunk = ""
            
            for i in range(0, len(sentences), 2):
                sentence = sentences[i]
                if i + 1 < len(sentences):
                    sentence += sentences[i + 1]
                
                if len(temp_chunk) + len(sentence) <= max_size:
                    temp_chunk += sentence
                else:
                    if temp_chunk:
                        chunks.append(temp_chunk.strip())
                    temp_chunk = sentence
            
            if temp_chunk:
                current_chunk = temp_chunk
        else:
            test_chunk = current_chunk + "\n\n" + paragraph if current_chunk else paragraph
            
            if len(test_chunk) <= max_size:
                current_chunk = test_chunk
            else:
                if current_chunk:
                    chunks.append(current_chunk.strip())
                current_chunk = paragraph
    
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

# テキストを分割
chunks = split_text_into_chunks(novel_text, MAX_CHUNK_SIZE)

print(f"📦 分割結果:")
print(f"   チャンク数: {len(chunks)}")
print()
for i, chunk in enumerate(chunks):
    preview = chunk[:50].replace('\n', ' ') + '...' if len(chunk) > 50 else chunk.replace('\n', ' ')
    print(f"   📄 チャンク {i+1}: {len(chunk):,}文字 | {preview}")

---
## セル6: 🎙️ TTS APIで音声生成

このセルを実行すると音声生成が開始されます。進捗バーで状況を確認できます。

In [ ]:
def generate_audio_chunk(text: str, chunk_index: int, output_dir: str, voice: str, model: str) -> str:
    """1つのテキストチャンクから音声ファイルを生成"""
    output_path = os.path.join(output_dir, f"chunk_{chunk_index:03d}.mp3")
    
    try:
        response = client.audio.speech.create(
            model=model,
            voice=voice,
            input=text,
            response_format="mp3"
        )
        response.stream_to_file(output_path)
        return output_path
    except Exception as e:
        raise RuntimeError(f"チャンク {chunk_index} の音声生成に失敗: {str(e)}")

def generate_all_audio(chunks: list, voice: str, model: str) -> list:
    """全てのチャンクから音声ファイルを生成（プログレスバー付き）"""
    temp_dir = tempfile.mkdtemp()
    audio_files = []
    
    total_chars = sum(len(chunk) for chunk in chunks)
    processed_chars = 0
    
    # ヘッダー情報
    display(HTML(f"""
    <div style='padding: 15px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                border-radius: 10px; color: white; margin-bottom: 15px;'>
        <h3 style='margin: 0;'>🎙️ 音声生成を開始</h3>
        <p style='margin: 5px 0;'>🎤 音声: {VOICE_OPTIONS[voice]}</p>
        <p style='margin: 5px 0;'>📀 モデル: {model}</p>
        <p style='margin: 5px 0;'>📊 総文字数: {total_chars:,}文字 / {len(chunks)}チャンク</p>
    </div>
    """))
    
    # プログレスバー
    progress_bar = tqdm(
        total=len(chunks),
        desc="🔊 音声生成中",
        unit="チャンク",
        bar_format='{l_bar}{bar:30}{r_bar}',
        colour='green'
    )
    
    # ステータス表示用
    status_widget = widgets.HTML()
    display(status_widget)
    
    start_time = time.time()
    
    for i, chunk in enumerate(chunks):
        try:
            audio_path = generate_audio_chunk(chunk, i + 1, temp_dir, voice, model)
            audio_files.append(audio_path)
            processed_chars += len(chunk)
            
            # 時間計算
            elapsed = time.time() - start_time
            avg_time = elapsed / (i + 1)
            remaining = avg_time * (len(chunks) - (i + 1))
            
            # ステータス更新
            status_widget.value = f"""
            <div style='padding: 10px; background-color: #f8f9fa; border-radius: 5px; 
                        border-left: 4px solid #28a745; margin-top: 10px;'>
                <b>✅ チャンク {i+1}/{len(chunks)} 完了</b> ({len(chunk):,}文字)<br>
                📊 進捗: {processed_chars:,}/{total_chars:,}文字 ({processed_chars/total_chars*100:.1f}%)<br>
                ⏱️ 経過: {int(elapsed//60)}分{int(elapsed%60)}秒 | 
                ⏳ 残り: 約{int(remaining//60)}分{int(remaining%60)}秒
            </div>
            """
            
            progress_bar.update(1)
            
            if i < len(chunks) - 1:
                time.sleep(REQUEST_INTERVAL)
                
        except Exception as e:
            progress_bar.close()
            print(f"\n❌ チャンク {i+1} でエラー: {str(e)}")
            raise
    
    progress_bar.close()
    
    total_time = time.time() - start_time
    display(HTML(f"""
    <div style='padding: 15px; background-color: #d4edda; border-radius: 10px; margin-top: 15px;'>
        <h3 style='margin: 0; color: #155724;'>🎉 音声生成完了！</h3>
        <p style='margin: 5px 0;'>✅ {len(audio_files)}チャンク生成</p>
        <p style='margin: 5px 0;'>⏱️ 総処理時間: {int(total_time//60)}分{int(total_time%60)}秒</p>
    </div>
    """))
    
    return audio_files

# 音声を生成
audio_files = generate_all_audio(chunks, TTS_VOICE, TTS_MODEL)

---
## セル7: 🔗 音声ファイルの結合とダウンロード

In [ ]:
def combine_audio_files(audio_files: list, output_path: str) -> str:
    """複数の音声ファイルを1つに結合"""
    
    display(HTML("""
    <div style='padding: 10px; background-color: #cce5ff; border-radius: 5px;'>
        <b>🔗 音声ファイルを結合しています...</b>
    </div>
    """))
    
    if not audio_files:
        raise ValueError("結合する音声ファイルがありません")
    
    combined = AudioSegment.from_mp3(audio_files[0])
    
    progress_bar = tqdm(
        total=len(audio_files),
        desc="🔗 結合中",
        unit="ファイル",
        bar_format='{l_bar}{bar:30}{r_bar}',
        colour='blue'
    )
    progress_bar.update(1)
    
    for audio_file in audio_files[1:]:
        segment = AudioSegment.from_mp3(audio_file)
        combined += segment
        progress_bar.update(1)
    
    progress_bar.close()
    
    print("💾 ファイルを保存中...")
    combined.export(output_path, format="mp3")
    
    file_size = os.path.getsize(output_path) / (1024 * 1024)
    duration = len(combined) / 1000
    
    display(HTML(f"""
    <div style='padding: 15px; background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%); 
                border-radius: 10px; color: white; margin-top: 15px;'>
        <h3 style='margin: 0;'>✅ 結合完了！</h3>
        <p style='margin: 5px 0;'>📁 ファイル名: {output_path}</p>
        <p style='margin: 5px 0;'>📊 ファイルサイズ: {file_size:.2f} MB</p>
        <p style='margin: 5px 0;'>⏱️ 再生時間: {int(duration // 60)}分 {int(duration % 60)}秒</p>
    </div>
    """))
    
    return output_path

# 音声を結合
OUTPUT_FILENAME = "novel_full.mp3"
final_audio_path = combine_audio_files(audio_files, OUTPUT_FILENAME)

# ダウンロード
print("\n📥 ダウンロードを開始します...")
files.download(final_audio_path)

display(HTML("""
<div style='padding: 20px; background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); 
            border-radius: 10px; color: white; text-align: center; margin-top: 20px;'>
    <h2 style='margin: 0;'>🎉 すべての処理が完了しました！</h2>
    <p style='margin: 10px 0 0 0;'>ダウンロードされた novel_full.mp3 をお楽しみください 🎧</p>
</div>
"""))

---
## セル8: 🧹 クリーンアップ（オプション）

一時ファイルを削除したい場合は、このセルを実行してください。

In [ ]:
# 一時ファイルを削除
for audio_file in audio_files:
    try:
        os.remove(audio_file)
    except:
        pass

print("🧹 一時ファイルを削除しました")